# I. Tweet Data Cleaning



In [1]:
## Tweet Table

In [29]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json
import time
import re
import datetime
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_colwidth', 25)

In [2]:
# function to cheack whether an object (data) contains attribute (x) or not
def check_var(data, x):
    try:
        data[x]
        return True
    except:
        return False

In [3]:
%%time
# making mongoDB database and collection
client = MongoClient('localhost', 27017)
db = client['mydatabase']

want_drop = input("Do you want to drop collection (yes/no): ")
if(want_drop == 'yes'): ## if type yes, then it will drop tweet table from mongodb to prepare new importing.
    db.tweet.drop()
    
tweet = db['tweet']

Do you want to drop collection (yes/no): no
Wall time: 980 ms


In [5]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [6]:
# yelin directory '/Users/yelin/Downloads/corona-out-3/corona-out-3'
# Kanya 'C:/Users/maxmo/OneDrive/Desktop/Rutgers/Spring2020/Data Management/corona-out-3'
# Hui Wang  file = '/Users/huiwang/Downloads/corona-out-3'
file = '/Users/huiwang/Downloads/corona-out-3'

In [7]:
%%time
# id, user_id, created_at, text, retweet_count, favorite_count, reply_count, hashtags
import json
with open(file, 'r', encoding = 'utf8') as f1 :
    for line in f1 :
        try :
            data = json.loads(line)
        except :
            continue
        
        hashtag = [] #to collect list of hashtags
        
        if (data['lang'] == 'en') :
            
            # case 1: check whether it is a retweet data or not
            if (check_var(data, 'retweeted_status')) :

                Created_at = pd.to_datetime(data['retweeted_status']['created_at'])
                UserID = data['retweeted_status']['user']['id']
                ID = data['retweeted_status']['id']
                retweet_count = data['retweeted_status']['retweet_count']
                favorite_count = data['retweeted_status']['favorite_count']
                reply_count = data['retweeted_status']['reply_count']

                # check whether there is extension version of that tweet or not        
                if (check_var(data['retweeted_status'], 'extended_tweet')) :
                    Text = data['retweeted_status']['extended_tweet']['full_text']
                    for t in data['retweeted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else :
                    Text = data['retweeted_status']['text']
                    for t in data['retweeted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

                # check whether it is retweet of quoted tweet or not
                if (check_var(data['retweeted_status'], 'quoted_status')) :

                    #check whether there is extension version of the original tweet or not 
                    if (check_var(data['retweeted_status']['quoted_status'], 'extended_tweet')) :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['extended_tweet']['full_text']
                        for t in data['retweeted_status']['quoted_status']['extended_tweet']['entities']['hashtags'] :
                            hashtag.append(t['text'])                
                    else :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['text']
                        for t in data['retweeted_status']['quoted_status']['entities']['hashtags'] :
                            hashtag.append(t['text'])               
 
            # case 2: check whether it is a quote tweet data or not
            elif (check_var(data, 'quoted_status')) :
                Created_at = pd.to_datetime(data['created_at'])
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                # check whether there is extension version of that quote tweet or not 
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text'] + ' || ' 
                else:
                    Text = data['text'] + ' || '

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

                # check whether there is extension version of the originl tweet or not 
                if (check_var(data['quoted_status'], 'extended_tweet')) :
                    Text = Text + data['quoted_status']['extended_tweet']['full_text']
                    for t in data['quoted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else:
                    Text = Text + data['quoted_status']['text']
                    for t in data['quoted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

            # if it does not belong to case 1 and case 2, it is an ordinary tweet data
            else :
                Created_at = pd.to_datetime(data['created_at'])
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                # check whether there is extension version of that tweet or not  
                if (check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text']
                else :
                    Text = data['text']

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

            tw = {'id' : ID,
                  'user_id' : UserID,
                  'created_at' : Created_at,
                  'text' : Text,
                  'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count,
                  'reply_count' : reply_count,
                  'hashtags': hashtag
                 }

            # if the data is already in our database, we update it 
            if (tweet.count_documents({'id' : tw['id']}) > 0) :
                db.tweet.update_one({'id' : tw['id']},
                                    {'$set' : {'retweet_count' : tw['retweet_count'],
                                               'favorite_count' : tw['favorite_count'],
                                               'reply_count' : tw['reply_count']}})
            # if it is not, we insert it    
            else :
                tweet.insert_one(tw)

CPU times: user 1min 37s, sys: 5.99 s, total: 1min 43s
Wall time: 29min 47s


'id_1'

In [4]:
# create indexes for tweet id, text, hashtags and created_at
tweet.create_index([ ('id', 1) ])
tweet.create_index([ ('text', 1) ])
tweet.create_index([ ('hashtags', 1) ])
tweet.create_index([ ('created_at', 1) ])

'created_at_1'

In [5]:
tweet_table = pd.DataFrame(tweet.find())
tweet_table

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea8a27017ef96e84cfa3565,1253949413191344128,207809313,2020-04-25 07:30:12,India’s war with Corona is ongoing.\n\nPlay your part and make sure no one around you starves.\n\nYour contribution to defeat hunger is as valuable as defeating Corona.\n\n#FeedTheNeedy https://t.co/TUWf2nqPgZ,398,2144,108,[FeedTheNeedy]
1,5ea8a27017ef96e84cfa3566,1253992905703862272,858859031464751104,2020-04-25 10:23:01,VIDEO | 25.04.20\n\n&gt; Monsta X appears on Yonhap News TV - Idol comeback through Corona... Turning on the Music industry.\n\n🔗https://t.co/kW5hsP5DeD\n\n#몬스타엑스 #MONSTAX\n#판타지아엑스 #FANTASIA_X\n@OfficialMonstaX\n\nhttps://t.co/tJN4Iyv1Gh https://t.co/GukJXWjaPW,59,144,1,"[몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]"
...,...,...,...,...,...,...,...,...,...
19074,5ea8a4bd17ef96e84cfa7fe7,1254059747738816512,3130596629,2020-04-25 14:48:37,We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump #ashamed \n@Arpx #arleg https://t.co/ELyI69q8pp,0,0,0,"[Corona, ashamed]"
19075,5ea8a4bd17ef96e84cfa7fe8,1254059751379361793,2279976427,2020-04-25 14:48:38,@VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona,0,0,0,[]






# II. User Data Cleaning





In [9]:
## User Table

In [6]:
import pandas as pd
import psycopg2
import json

In [67]:
## Creating the database
conn = psycopg2.connect("dbname=postgres port=5432 user=postgres password=password")
conn.set_session(autocommit = True)
cur = conn.cursor()

cur.execute("CREATE DATABASE twitter_users" )
conn.commit()
cur.close()
conn.close()

DuplicateDatabase: database "twitter_users" already exists


In [68]:
# Creating the table
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()
#cur.execute("drop table users_data")
cur.execute("CREATE TABLE IF NOT EXISTS users_data( user_id bigint PRIMARY KEY, name varchar(255), user_name varchar(255), verified_status boolean,followers_count integer,friends_count integer,statuses_count integer,user_location varchar(500),favourites_count integer);" )
conn.commit()
cur.close()
conn.close()

In [69]:
def check_var(data,x):
    try:
        data[x]
        return True
    except:
        return False

In [36]:
%%time
# This function takes the the variable inputs and stores it into a PostgreSQL database
def store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count):
    conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
    cur = conn.cursor()
    insert_query = "INSERT INTO users_data (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET name=EXCLUDED.name, user_name=EXCLUDED.user_name, verified_status=EXCLUDED.verified_status, followers_count=EXCLUDED.followers_count, friends_count=EXCLUDED.friends_count, statuses_count=EXCLUDED.statuses_count, user_location=EXCLUDED.user_location, favourites_count=EXCLUDED.favourites_count"
    cur.execute(insert_query, (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location,favourites_count))
    conn.commit()
    cur.close()
    conn.close()
    
    
# Decoding the JSON file
with open(file, 'r', encoding = 'utf8') as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
        except:
            continue
        
        if (data['lang'] == 'en'): # only 
            
            if (check_var(data, 'retweeted_status')):

                user_id = data['retweeted_status']['user']['id']
                name = data['retweeted_status']['user']['name']
                user_name = data['retweeted_status']['user']['screen_name']
                verified_status = data['retweeted_status']['user']['verified']
                followers_count = data['retweeted_status']['user']['followers_count']
                friends_count = data['retweeted_status']['user']['friends_count']
                statuses_count = data['retweeted_status']['user']['statuses_count']
                user_location = data['retweeted_status']['user']['location']
                favourites_count= data['retweeted_status']['user']['favourites_count']


            else :
                user_id = data['user']['id']
                name = data['user']['name']
                user_name = data['user']['screen_name']
                verified_status = data['user']['verified']
                followers_count = data['user']['followers_count']
                friends_count = data['user']['friends_count']
                statuses_count = data['user']['statuses_count']
                user_location = data['user']['location']
                favourites_count= data['user']['favourites_count']


            # insert the data into the PostgreSQL database
            store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count)

CPU times: user 45.7 s, sys: 17 s, total: 1min 2s
Wall time: 12min 27s


In [7]:
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()


# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [8]:
user_table = create_pandas_table("SELECT * FROM users_data")
user_table

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,375777294,TeéLaneeë🌺,TWD40_,False,637,408,18976,None,1325
1,1132273796138905600,Terri Kamp,RampTheresa,False,1522,256,20861,None,29166
...,...,...,...,...,...,...,...,...,...
15671,734263483173212160,dirty d 😽💙,deannarae21,False,684,482,43705,baerobbie 💙,75847
15672,2279976427,Venugopi,Venu7630,False,3,35,42,None,83






# III. Searching Application





Implement 1. Top 3 Popular Original Tweet

In [81]:
%%time
# using MongoDB
# Top 3 popular original tweet ranked by the retweet_count, reply_count, and favorite_count 
pop_tweet = pd.DataFrame(tweet.find({},{'user_id':1,'created_at': 1,'text':1,'retweet_count':1,'favorite_count':1,
                                         'reply_count':1,'hashtags':1}).sort([("retweet_count",-1),("reply_count",-1),
                                                                              ("favorite_count",-1)]).limit(10))
pop_tweet.head(3)

CPU times: user 6.11 ms, sys: 12.5 ms, total: 18.6 ms
Wall time: 176 ms


,_id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea8cf6a82a63f13f9692cfe,1100261477989126145,2020-03-13 00:43:40,*corona virus enters my body*\n\nThe 4 Flintstone gummies I ate in 2005: https://t.co/3STfdIQtaT,237307,811062,1811,[]
1,5ea8ced182a63f13f969264d,1112592502727548928,2020-03-18 17:51:18,When this Corona shit passes we have to promise each other that we’re going to tell our kids that we survived a zombie apocalypse in 2020,181584,764405,1595,[]
2,5ea8cde182a63f13f9691acb,1131227186,2020-03-10 17:52:14,THIS MAN IS A GENIUS he figured out the Corona virus problem 😮 https://t.co/EZP7IqTtxV,179037,515867,1578,[]


Implement 2. Top 10 Popular Hashtag

In [78]:
%%time
# using MongoDB
# TOP 10 popular hashtag ranked by aggregating the lists of hashtags (case insensitive)
from bson.son import SON
pipeline = [
     {"$unwind": "$hashtags"},
     {"$group": {"_id": {"$toLower": "$hashtags"}, "count": {"$sum": 1}}},
     {"$sort": SON([("count", -1), ("_id", -1)])}
] 
import pprint
pprint.pprint(list(db.tweet.aggregate(pipeline))[0:10])

[{'_id': 'corona', 'count': 1248},
 {'_id': 'covid19', 'count': 542},
 {'_id': 'coronavirus', 'count': 384},
 {'_id': 'covid_19', 'count': 292},
 {'_id': 'lockdown', 'count': 166},
 {'_id': 'covid', 'count': 129},
 {'_id': 'indiafightscorona', 'count': 119},
 {'_id': 'संतरामपालजी_का_सत्संग_सुनें', 'count': 114},
 {'_id': 'stayhome', 'count': 102},
 {'_id': 'covidー19', 'count': 72}]
CPU times: user 17 ms, sys: 2.29 ms, total: 19.3 ms
Wall time: 136 ms


Implement 3. Top Active Verified User & Top Popular Verified User

In [82]:
# using SQL
# Top 3 active verified users ranked by statuses_count
active_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY statuses_count DESC LIMIT 10")
active_user.head(3)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,47596019,Liputan6.com,liputan6dotcom,True,3687560,693,1381273,Jakarta Indonesia,7062
1,16973333,The Independent,Independent,True,3202840,1146,976895,"London, England",3
2,378809160,𝔾𝕀𝔻𝕀𝕋ℝ𝔸𝔽𝔽𝕀ℂ,Gidi_Traffic,True,1676895,5826,922265,Everywhere,14478


In [83]:
# Top 3 popular verified users ranked by followers_count
pop_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY followers_count DESC LIMIT 10")
pop_user.head(3)


,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,18839785,Narendra Modi,narendramodi,True,55786179,2364,26559,India,0
1,807095,The New York Times,nytimes,True,46361159,904,393823,New York City,18483
2,145125358,Amitabh Bachchan,SrBachchan,True,41596464,1833,66428,"Mumbai, India",75


Implement 4. Top Active Location

In [47]:
# using SQL
# Top 3 active location ranked by tweets count
pop_location = create_pandas_table("SELECT user_location, count(*) as amount FROM users_data WHERE user_location != 'None' GROUP BY user_location ORDER BY amount DESC LIMIT 10")
pop_location.head(3)

,user_location,amount
0,India,363
1,"New Delhi, India",185
2,United States,125


Implement 5. Top Earliest Original Tweet

In [11]:
# define function for joining with User table 
def result_with_user (tweet):
    s_user = tuple(tweet["user_id"].tolist())
    params = {'l': s_user}

    cur.execute('SELECT * FROM users_data where user_id in %(l)s', params)
    search_user = pd.DataFrame(cur.fetchall())
    search_user.columns = [i[0] for i in cur.description]

    search_full = pd.merge(tweet, search_user, how = 'left', on = ['user_id']).drop(['_id', 'id', 'user_id'], axis = 1)
    return search_full


In [30]:
%%time
# Top earliest created original tweet that is still getting retweeted in the time slot.

early_top = pd.DataFrame(tweet.find().sort([("created_at",1)]).limit(3))
result_with_user(early_top)

Wall time: 107 ms


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,2013-06-03 15:32:48,Corona virus....its c...,73776,131225,9968,[],Marco,Marco_Acortes,False,5652,106,3839,None,4842
1,2017-06-08 16:14:48,But to return to a mo...,16,39,0,[],coldplums,coldplums,False,4341,1435,24661,on the road,41167
2,2020-01-25 18:30:29,I bet the marketing p...,306,797,23,[],Rick Aaron,RickAaron,False,12391,12854,78740,None,138699


Text, hashtag, and time search

In [150]:
import time
import datetime
import timeit
from collections import OrderedDict

# set for storing keyword
stored_keyword = set()
stored_hashtag = set()
stored_date = set()
# dictionary of dictionaries
dict_text = OrderedDict()
dict_hashtag = OrderedDict()
dict_date = OrderedDict()


# function: store dataframe in dictionary 
def put_into_cache(keyword, df, dict_set) :
    if (len(dict_set) == 10): # we can hold 10 caches if it over, we deleted oldest one
        del dict_set[list(dict_set.keys())[0]] 
    
    dict_set[keyword] = df

In [151]:
# function: read cache from dictionary to dataframe
import copy
def delete_key(keyword,dict_set):
    del dict_set[keyword]

def dict_to_df(keyword, dict_set) :
    dict_set.move_to_end(keyword)
    return pd.DataFrame.from_dict(dict_set[keyword])

In [152]:
def text_search():
    user_keyword = input("Please enter a keyword: ")
    start = timeit.default_timer()
  
    regx = re.compile(user_keyword, re.IGNORECASE)
    
    if (user_keyword.lower() not in stored_keyword) :
        search_tweet = pd.DataFrame(
            tweet.find({'text': {"$regex": regx}}).sort([("retweet_count", -1), ("reply_count", -1), ("favorite_count", -1)])
        )

        if (len(search_tweet) == 0) :
            print("Error: No result for %s" %user_keyword)
            search_full = None

        else :
            search_full = result_with_user(search_tweet)
            
            #put into cache
            stored_keyword.add(user_keyword.lower())
            put_into_cache(user_keyword.lower(), search_full, dict_text)

    else :
        search_full = dict_to_df(user_keyword.lower(), dict_text)    

    stop = timeit.default_timer()
    
    print('Running Time: ', "%.3f second" %(stop - start))
    return search_full

In [30]:
def hashtag_search():
    user_keyword = input("Please enter a keyword: ")
    start = timeit.default_timer()
    
    if (user_keyword.lower() not in stored_hashtag) :
        # uses the index to find
        hashtag_tweet = pd.DataFrame(
            db.tweet.find({"hashtags": {"$in": [user_keyword]}}).collation( { "locale": 'en', "strength": 1 } ).sort([("retweet_count", -1), ("reply_count", -1), ("favorite_count", -1)])
        )

        if (len(hashtag_tweet) == 0) :
            print("Error: No result for %s" %user_keyword)
            hash_search_table = None

        else :
            hash_search_table = result_with_user(hashtag_tweet)
            
            #put into cache
            stored_hashtag.add(user_keyword.lower())
            put_into_cache(user_keyword.lower(), hash_search_table, dict_hashtag)

    else :
        hash_search_table = dict_to_df(user_keyword.lower(), dict_hashtag)    
    
    stop = timeit.default_timer()
    print('Running Time: ', "%.3f second" %(stop - start))
    return hash_search_table

In [31]:
def time_search():
    period = True
    while period :
        dateinput1 = input("Please enter the start date (format: yyyy-m-d (ex: 2020-4-13)): ")
        x1 = [int(x) for x in dateinput1.split("-")]
        dateinput2 = input("Please enter the end date (format: yyyy-m-d (ex: 2020-4-15)): ")
        x2 = [int(x) for x in dateinput2.split("-")]

        start = datetime.datetime(x1[0], x1[1], x1[2], 0, 0, 0, 0)
        end = datetime.datetime(x2[0], x2[1], x2[2], 23, 59, 59, 0)

        if (start <= end) :
            period = False
        else :
            print("Error! Try again. Start date should be earlier than end date.")
            
    start_time = timeit.default_timer()
    
    if ((dateinput1 + " to " + dateinput2) not in stored_date) :
    
        ##sort it by time (asc)
        range_tweet = pd.DataFrame(tweet.find({'created_at': {"$gte": start, "$lte": end}}).sort([("created_at", 1)]))

        if (len(range_tweet) == 0) :
            print("Error: No result for input range!")
            range_search_table = None
        else :
            range_search_table = result_with_user(range_tweet)

            stored_date.add(dateinput1 + " to " + dateinput2)
            put_into_cache(dateinput1 + " to " + dateinput2, range_search_table, dict_date)

    else :
        range_search_table = dict_to_df(dateinput1 + " to " + dateinput2, dict_date)            
    
    stop_time = timeit.default_timer()
    print('Running Time: ', "%.3f second" %(stop_time - start_time))
    return range_search_table

In [32]:
text_search()

Please enter a keyword: hi
Running Time:  0.435 second


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,2020-03-18 17:51:18,When this Corona shit...,181584,764405,1595,[],D i l l 🦦,Khydill,False,3862,532,7096,Quarantined,7615
1,2020-03-10 17:52:14,THIS MAN IS A GENIUS ...,179037,515867,1578,[],Carlitos 🦉♛,_AyeeCarlos_,False,1345,1122,10367,"San Diego, CA 🌴",35200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9477,2020-04-25 14:48:36,The good news! He can...,0,0,0,[],'Mr. Right',realtrumpster01,False,7510,8123,172794,None,17887
9478,2020-04-25 14:48:38,@VSReddy_MP I think w...,0,0,0,[],Venugopi,Venu7630,False,3,35,42,None,83


In [33]:
hashtag_search()

Please enter a keyword: coRona
Running Time:  0.000 second


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,2020-03-26 09:34:16,The Govt announcement...,14083,83004,5500,[Corona],Rahul Gandhi,RahulGandhi,True,13497949,253,4610,"12, Tughlak Lane, New...",359
1,2020-03-22 05:52:25,Can you see the link?...,6346,35558,789,"[JantaCurfew, India, ...",Narendra Modi,narendramodi,True,55786179,2364,26559,India,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,2020-04-25 14:48:23,Really sucks to see p...,0,0,0,"[corona, covid19, chi...",瘋Insanity 🦋,brycebcool,False,190,2472,34225,None,43519
1238,2020-04-25 14:48:37,We have 1/4 of all de...,0,0,0,"[Corona, ashamed]",Arkansas Boy,ArkansasWatch,False,276,867,4411,#Ark #LittleRock,1128


In [28]:
time_search()

Please enter the start date (format: yyyy-m-d (ex: 2020-4-13)): 2020-4-16
Please enter the end date (format: yyyy-m-d (ex: 2020-4-15)): 2020-4-30
Running Time:  0.781 second


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,2020-04-16 02:14:22,#GodMorningThursday\nOnly real lord Kabir\nCan Save Us From All Difficulties\nLike Corona \nPeople From All The World \nMust Know The Supreme God\n#MustListen_Satsang of \nSaint Rampal Ji Maharaj https://t.co/Nz9fzAfyNd,25,26,0,"[GodMorningThursday, MustListen_Satsang]",Rahul Singh D,RahulDasawat,False,2138,1567,19050,None,22270
1,2020-04-16 02:54:50,"Stay at home means stay at home, even if it means I have to carry you home 😂\n#StayAtHome\n#Corona https://t.co/0Cz316FSZX",24,8,1,"[StayAtHome, Corona]",🔥🔥 Leher56 🔥🔥,Leher56,False,859,218,63123,"Jharkhand, India",2567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18619,2020-04-25 14:48:37,We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump #ashamed \n@Arpx #arleg https://t.co/ELyI69q8pp,0,0,0,"[Corona, ashamed]",Arkansas Boy,ArkansasWatch,False,276,867,4411,#Ark #LittleRock,1128
18620,2020-04-25 14:48:38,@VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona,0,0,0,[],Venugopi,Venu7630,False,3,35,42,None,83
